# **Building an AI Financial Analyst with the Assistants API**

* in this guide, we discuss how to build an AI financial analyst using the Assistants API, function calling, and Code Interpreter.
  
  ### **🔗 [Source](https://www.mlq.ai/ai-financial-analyst-assistants-api/)**

## **Steps to Create a Financial Analyst Assistant**

At a high-level, the steps we need to take to create an Assistant are:

 1. **Defining financial functions:** We need to define the Financial Modeling Prep API calls to retrieve financial
   
 2. **Defining the Assistant:** Our AI agent will use models and instructions alongside the function calling tool to retrieve & analyze financial data.

 3. **Creating a Thread:** A thread is essentially a conversation flow where we can feed the agent with user queries, function call responses, and assistant replies.
   
 4. **Adding Messages:** We then input user queries to the Thread in the form of Messages.
   
 5. **Running the Assistant:** We then create a Run, which is the Assistant processing the Thread, calling necessary tools, and generating appropriate responses.

## **Step 0:** Setting up the Environment

In [78]:
import os
# from dotenv import find_dotenv, load_dotenv
import json

from openai import OpenAI
import time
# API keys are stored in .env file
# _:bool=load_dotenv(find_dotenv()) 
# Load environment variables
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] 
FMP_API_KEY = os.environ["FMP_API_KEY"]

# Create an OpenAI object
client = OpenAI()


## **Step 1:** Defining Financial Functions

In [ ]:
import requests

def get_income_statement(ticker):
    """Get income statement for a given ticker and period"""
    # Construct the URL with the provided parameters
    url = f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={FMP_API_KEY}'
   
    # Make the HTTP GET request
    response = requests.get(url)

    # Check if the response contains valid JSON data
    if response.status_code == 200:  # Assuming 200 means a successful response
        try:
            # Attempt to parse the JSON content
            json_data = json.dumps(response.json())
            return json_data
        except Exception as e:
            print(f"Error parsing JSON: {e}")
            return None
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None

def get_balance_sheet(ticker):
    """get balance sheet"""
    
    # construct the URL with the provided parameters
    url=f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={FMP_API_KEY}"
    
    # make the HTTP GET request
    response = requests.get(url)
    
    # check if the respone contains valid JSON data
    if response.status_code == 200:
        try:
            # Attempt to parse the JSON content
            json_data = json.dumps(response.json())
            return json_data
        except Exception as e:
            print(f"Error parsing JSON: {e}")
            return None  
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None

def get_cash_flow_statement(ticker):
    """get the cash flow statement"""
    
    # construct the URL with the provided parameters
    url=f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}"
    
    # make the HTTP GET request
    response= requests.get(url)
    
    # check if the respone contains valid JSON data
    if response.status_code == 200:
        try:
            # Attempt to parse the JSON content
            json_data = json.dumps(response.json())
            return json_data
        except Exception as e:
            print(f"Error parsing JSON: {e}")
            return None  
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None 
def get_key_matrics(ticker):
    """get the key matrics"""
    
    url=f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?apikey={FMP_API_KEY}" 
    
    response =requests.get(url)   
    
    if response.status_code == 200:
        try:
            json_data = json.dumps(response.json())
            return json_data
        except Exception as e:
            print(f"Error parsing JSON: {e}")
            return None
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None    
            
def get_financial_ratios(ticker):
    """get the financial ratios"""  
    
    url=f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={FMP_API_KEY}"    
    
    #  http request 
    response= requests.get(url)
    
    if response.status_code == 200:
        try:
            json_data = json.dumps(response.json())
            return json_data
        except Exception as e:
            print(f"Error parsing JSON: {e}")
            return None    
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None   
def get_financial_growth(ticker):
    """get the financial growth"""
    
    url =  f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?apikey={FMP_API_KEY}"
    # makes http requests
    response = requests.get(url)     
    
    if response.status_code == 200:
        try:
            json_data = json.dumps(response.json())
            print(json_data)
            return json_data
        except Exception as e:
            print(f"Error parsing JSON: {e}")
            return None
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None     
  

## **Step 2:** Map available functions

In [80]:
available_functions={
    "get_income_statement":get_income_statement,
    "get_balance_sheet":get_balance_sheet,
    "get_cash_flow_statement":get_cash_flow_statement,
    "get_key_matrics":get_key_matrics,
    "get_financial_ratios":get_financial_ratios,
    "get_financial_growth":get_financial_growth
}

## **Step 3:** Creating the Assistant

In [81]:
# Define run_assistant(user_message)
def run_assistant(user_message: str):
    # Creating an assistant with specific instructions and tools
    assistant = client.beta.assistants.create(
        name="Financial Assistant",
        description="This assistant will help you with financial data",
        instructions="Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends.",

       model="gpt-3.5-turbo-1106",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_income_statement",
                    "description": "Get the financial income statement of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_balance_sheet",
                    "description": "Get the financial balance sheet of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            },

            {
                "type": "function",
                "function": {
                    "name": "get_cash_flow_statement",
                    "description": "Get the financial cash flow statement of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_key_metrics",
                    "description": "Get the financial key metrics of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_financial_ratios",
                    "description": "Get the financial ratios of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_financial_ratios",
                    "description": "Get the financial ratios of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_financial_ratios",
                    "description": "Get the financial ratios of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_financial_growth",
                    "description": "Get the financial growth data of a company",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "Company-share names listed in the stock market (e.g., Apple Inc. = AAPL)",
                            }
                        },
                        "required": ["ticker"],
                    },
                }
            }
        ]
    )
    """Step 4 -> Initiating a Thread"""
    # Creating a new thread
    thread = client.beta.threads.create()
    print(thread.id)

    """Step 5 -> Adding Messages to the Thread"""
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )
    """Step 6 -> Running and Monitoring the Assistant"""
    # Start a run
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id, run_id=run.id)

        # Add run steps retrieval here
        run_steps = client.beta.threads.runs.steps.list(
            thread_id=thread.id, run_id=run.id)
        print("Run Steps:", run_steps)

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**function_args)
        
                    # Print or log the output for debugging
                    print(f"Output for {function_name}: {output}")
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run.status == "completed":
            # List the messages to get the response
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            # Print the user's message first
            for message in messages.data:
                if message.role == "user":
                    role_label = "User"
                    message_content = message.content[0].text.value
                    print(f"{role_label}: {message_content}\n")
            for message in messages.data:
                if message.role == "assistant":
                    role_label = "Assistant"
                    message_content = message.content[0].text.value
                    print(f"{role_label}: {message_content}\n")       
                          
            break 
            # Exit the loop after processing the completed run
        elif run.status == "failed":
            print("Run failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break

In [ ]:
run_assistant("what is the financial growth AAPl")

## User : what is the financial growth AAPl

**Assistant:** The financial growth data for AAPL (Apple Inc.) shows the following key growth metrics over the last few years:

- Revenue Growth:
  - 2022: 7.79%
  - 2021: 33.26%
  - 2020: 5.51%
  - 2019: -2.04%
  - 2018: N/A

- Net Income Growth:
  - 2022: 5.41%
  - 2021: 64.92%
...
  - 2018: N/A

These growth metrics demonstrate a mix of positive and negative growth rates across different financial indicators, reflecting various aspects of the company's performance over the years.